In [307]:
import pandas as pd
import numpy  as np

In [441]:
index_df = pd.read_excel(r'../data/CFS_AREA_2017_county.xlsx')
index_df = index_df[['state_FIPS','county_FIPS','FIPS','county','FAF']]
index_df.rename(columns={'state_FIPS':'fipstate','county_FIPS':'fipscty'},inplace=True)



res_df = pd.read_excel(r'../data/fossil_fuel/gas_consumption/residential_gas_consumption.xls','Data 1',skiprows=2)
res_df.columns = res_df.columns.str.removesuffix(' Natural Gas Residential Consumption (MMcf)')
res_df['year'] = pd.DatetimeIndex(res_df['Date']).year
res_df.drop(['Date','U.S.'],axis=1,inplace=True)
res_df = res_df.set_index('year').T[[2017,2018,2019]]
res_df = res_df.T.unstack().reset_index()
res_df.rename({'level_0' : 'state',0:'residential gas consumption'},inplace=True,axis='columns')



com_df = pd.read_excel(r'../data/fossil_fuel/gas_consumption/commercial_gas_consumption.xls','Data 1',skiprows=2)
com_df.columns = com_df.columns.str.removeprefix('Natural Gas Deliveries to Commercial Consumers (Including Vehicle Fuel through 1996) in ')
com_df.columns = com_df.columns.str.removesuffix(' (MMcf)')
com_df['year'] = pd.DatetimeIndex(com_df['Date']).year
com_df.drop(['Date','the U.S.'],axis=1,inplace=True)
com_df = com_df.set_index('year').T[[2017,2018,2019]]
com_df = com_df.T.unstack().reset_index()
com_df.rename({'level_0' : 'state',0:'commercial gas consumption'},inplace=True,axis='columns')
coms = ['42----','44----','45----','48----','49----','51----','52----','53----','54----',\
        '55----','56----','61----','62----','71----','72----','81----']
cbp_com = cbp_df[cbp_df.naics.isin(coms)]
temp = pd.merge(cbp_com,index_df,how='left',on=['fipstate','fipscty'])
temp.dropna(subset=['FAF'],inplace=True)
st_sum = temp.groupby(['fipstate','year'])['ap'].sum().reset_index()
st_sum.rename(columns={'ap':'ap_state'},inplace=True)
cbp_com = pd.merge(temp,st_sum,how='left',on=['fipstate','year'])
cy_sum = temp.groupby(['fipstate','fipscty','year'])['ap'].sum().reset_index()
cy_sum.rename(columns={'ap':'ap_county'},inplace=True)
cbp_com = pd.merge(cbp_com,cy_sum,how='left',on=['fipstate','fipscty','year'])
cbp_com = cbp_com[['year','county','FIPS','fipstate','est','emp','ap','ap_state','ap_county']]
cbp_com['proportion'] = cbp_com.ap_county/cbp_com.ap_state
states = pd.read_csv(r'../data/ZIP-COUNTY-FIPS_2017-06.csv')
states['FIPS'] = states['FIPS'].values.astype(str)
states['FIPS'] = states['FIPS'].str.zfill(5)
states['fipstate'] = states.FIPS.str[:2]
states = states[['state','fipstate']]
states['fipstate'] = states['fipstate'].values.astype(int)
com_df = pd.merge(com_df,states,how='left',on=['state'])
com_df.drop_duplicates(inplace=True)



ind_df = pd.read_excel(r'../data/fossil_fuel/gas_consumption/industrial_gas_consumption.xls','Data 1',skiprows=2)
ind_df.columns = ind_df.columns.str.removesuffix(' Natural Gas Industrial Consumption (MMcf)')
ind_df['year'] = pd.DatetimeIndex(ind_df['Date']).year
ind_df.drop(['Date','U.S.'],axis=1,inplace=True)
ind_df = ind_df.set_index('year').T[[2017,2018,2019]]
ind_df = ind_df.T.unstack().reset_index()
ind_df.rename({'level_0' : 'state',0:'industrial gas consumption'},inplace=True,axis='columns')
coms2 = ['21----','23----','31----','32----','33----']
cbp_ind = cbp_df[cbp_df.naics.isin(coms2)]
temp = pd.merge(cbp_ind,index_df,how='left',on=['fipstate','fipscty'])
temp.dropna(subset=['FAF'],inplace=True)
st_sum = temp.groupby(['fipstate','year'])['ap'].sum().reset_index()
st_sum.rename(columns={'ap':'ap_state'},inplace=True)
cbp_ind = pd.merge(temp,st_sum,how='left',on=['fipstate','year'])
cy_sum = temp.groupby(['fipstate','fipscty','year'])['ap'].sum().reset_index()
cy_sum.rename(columns={'ap':'ap_county'},inplace=True)
cbp_ind = pd.merge(cbp_ind,cy_sum,how='left',on=['fipstate','fipscty','year'])
cbp_ind = cbp_ind[['year','county','FIPS','fipstate','est','emp','ap','ap_state','ap_county']]
cbp_ind['proportion'] = cbp_ind.ap_county/cbp_ind.ap_state
ind_df = pd.merge(ind_df,states,how='left',on=['state'])
ind_df.drop_duplicates(inplace=True)



ele_df = pd.read_excel(r'../data/fossil_fuel/gas_consumption/electric_use_gas_consumption.xls','Data 1',skiprows=2)
ele_df.columns = ele_df.columns.str.removesuffix(' Natural Gas Deliveries to Electric Power Consumers (MMcf)')
ele_df['year'] = pd.DatetimeIndex(ele_df['Date']).year
ele_df.drop(['Date','U.S.'],axis=1,inplace=True)
ele_df = ele_df.set_index('year').T[[2017,2018,2019]]
ele_df = ele_df.T.unstack().reset_index()
ele_df.rename({'level_0' : 'state',0:'electrical use gas consumption'},inplace=True,axis='columns')

ele_cnty_17 = pd.read_excel(r'../data/fossil_fuel/gas_consumption/2017_final_revision.xlsx',\
                            'Page 5 Fuel Receipts and Costs',skiprows=4)

ele_cnty_18 = pd.read_excel(r'../data/fossil_fuel/gas_consumption/2018_final_revision.xlsx',\
                            'Page 5 Fuel Receipts and Costs',skiprows=4)
ele_cnty_19 = pd.read_excel(r'../data/fossil_fuel/gas_consumption/2019_final_revision.xlsx',\
                            'Page 5 Fuel Receipts and Costs',skiprows=4)
coms3 = ['22----']
cbp_ele = cbp_df[cbp_df.naics.isin(coms3)]
ele_cnty_df = pd.concat([ele_cnty_17,ele_cnty_18,ele_cnty_19])
ele_cnty_df = ele_cnty_df[ele_cnty_df.ENERGY_SOURCE=='NG']
ele_cnty_df = ele_cnty_df[['YEAR','MONTH','Plant Id','Plant Name','FUEL_COST','QUANTITY']]
summ = ele_cnty_df.groupby(['YEAR','Plant Id'])['QUANTITY'].sum().reset_index()
plant_df = pd.read_csv(r'../data/fossil_fuel/gas_consumption/power plant with county fips.csv')
plant_df = plant_df[['Plant Id','FIPS']]
ele_cnty_df.rename(columns={'YEAR':'year'},inplace=True)
temp = pd.merge(ele_cnty_df,plant_df,how='left',on='Plant Id')
cy_sum = temp.groupby(['FIPS','year'])['QUANTITY'].sum().reset_index()
cy_sum.rename(columns={'QUANTITY':'county_consumption'},inplace=True)
cbp_ele['fipscty'] = cbp_ele['fipscty'].values.astype(int)
cbp_ele['fipscty'] = cbp_ele['fipscty'].values.astype(str)
cbp_ele['fipscty'] = cbp_ele['fipscty'].str.zfill(3)

cbp_ele['fipstate'] = cbp_ele['fipstate'].values.astype(int)
cbp_ele['fipstate'] = cbp_ele['fipstate'].values.astype(str)
cbp_ele['fipstate'] = cbp_ele['fipstate'].str.zfill(2)

cbp_ele['FIPS'] = cbp_ele['fipstate']+cbp_ele['fipscty']
cbp_ele['FIPS'] = cbp_ele['FIPS'].astype(int)



pop_df = pd.read_excel(r'../data/county_pop.xlsx',skiprows=3)
pop_df.rename({'Unnamed: 0' : 'County'},inplace=True,axis='columns')
pop_df = pop_df.iloc[1: , :]
pop_df.County = pop_df.County.str.replace('.','')
pop_df = pop_df[['County',2017,2018,2019]]
pop_df = pop_df.melt(id_vars='County', var_name='year').sort_values('County')
pop_df['county'] = pop_df.County.str.split(',').str[0]
pop_df['state'] = pop_df.County.str.split(', ').str[1]
pop_df.drop('County',axis=1,inplace=True)
temp = pop_df.groupby(['state','year'])['value'].sum().reset_index()
temp.rename({'value':'state_value'},axis=1,inplace=True)
pop_df = pd.merge(pop_df,temp,how='left',on=['state','year'])
pop_df['proportion'] = pop_df.value/pop_df.state_value

cbp17 = pd.read_csv(r'../data/cbp/cbp17co.txt')
cbp17['year'] = 2017
cbp18 = pd.read_csv(r'../data/cbp/cbp18co.txt')
cbp18['year'] = 2018
cbp19 = pd.read_csv(r'../data/cbp/cbp19co.txt')
cbp19['year'] = 2019
cbp_df = pd.concat([cbp17,cbp18,cbp19])
cbp_df['year'] = cbp_df['year'].astype("Int64")



C:\Users\33o\AppData\Local\Temp\ipykernel_20152\3152805937.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbp_ele['fipscty'] = cbp_ele['fipscty'].values.astype(int)
C:\Users\33o\AppData\Local\Temp\ipykernel_20152\3152805937.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbp_ele['fipscty'] = cbp_ele['fipscty'].values.astype(str)
C:\Users\33o\AppData\Local\Temp\ipykernel_20152\3152805937.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [453]:
residential_df = pd.merge(pop_df,res_df,how='left',on=['state','year'])
residential_df['county_consumption'] = residential_df['proportion']*residential_df['residential gas consumption']
residential_df = pd.merge(residential_df,index_df,how='left',on=['county'])
residential_df = residential_df[['year','FIPS','county_consumption']]

commercial_df = pd.merge(cbp_com,com_df,how='left',on=['fipstate','year'])
commercial_df['county_consumption'] = commercial_df['proportion']*commercial_df['commercial gas consumption']
commercial_df = commercial_df[['year','FIPS','emp','est','ap','county_consumption']]

industrial_df = pd.merge(cbp_ind,ind_df,how='left',on=['fipstate','year'])
industrial_df['county_consumption'] = industrial_df['proportion']*industrial_df['industrial gas consumption']
industrial_df = industrial_df[['year','FIPS','emp','est','ap','county_consumption']]

electrical_df = pd.merge(cbp_ele,cy_sum,how='left',on=['year','FIPS'])
electrical_df = electrical_df[['year','FIPS','emp','est','ap','county_consumption']]

In [462]:
master_temp = pd.concat([residential_df,commercial_df,industrial_df,electrical_df])
master_table = master_temp.groupby(['year','FIPS']).sum().reset_index()
master_table.corr()

,year,FIPS,county_consumption,emp,est,ap
year,1.000000,0.000654,0.017191,0.003366,0.000788,0.008468
FIPS,0.000654,1.000000,-0.019759,-0.042625,-0.053716,-0.040198
county_consumption,0.017191,-0.019759,1.000000,0.417693,0.422511,0.353088
emp,0.003366,-0.042625,0.417693,1.000000,0.977375,0.956729
est,0.000788,-0.053716,0.422511,0.977375,1.000000,0.908505
ap,0.008468,-0.040198,0.353088,0.956729,0.908505,1.000000
